In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'


# Change Digits here

In [3]:
TRAINING_SIZE = 30000
DIGITS = 5

REVERSE = True

MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+- '

RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1


In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)


In [5]:
ctable = CharacterTable(chars)


# Data Generation

In [6]:
questions = []
expected = []
seen = set()

print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                            for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    
    # if operator == 1 do adder, if operator == 2 do substractor
    operator = np.random.choice(list('12'))
    if operator == '1' :
        
        # Pad the data with spaces such that it is always MAXLEN.
        q = '{}+{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        
        ans = str(a + b)
        # Answers can be of maximum size DIGITS + 1.
        ans += ' ' * (DIGITS + 1 - len(ans))
        
    elif operator == '2' :
        # Pad the data with spaces such that it is always MAXLEN.
        q = '{}-{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        
        ans = str(a - b)
        # Answers can be of maximum size DIGITS + 1.
        ans += ' ' * (DIGITS + 1 - len(ans))

    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))



Generating data...
Total addition questions: 30000


In [7]:
print(questions[:10], '\n' ,expected[:10])


['346+9350   ', '7+97       ', '80+3362    ', '833+92     ', '14-6364    ', '1-98       ', '17-89      ', '90-75      ', '679-94     ', '6-42       '] 
 ['9696  ', '104   ', '3442  ', '925   ', '-6350 ', '-97   ', '-72   ', '15    ', '585   ', '-36   ']


# Processing

In [8]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)


Vectorization...


In [9]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:25000]
train_y = y[:25000]
test_x = x[25000:]
test_y = y[25000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)



Training Data:
(22500, 11, 13)
(22500, 6, 13)
Validation Data:
(2500, 11, 13)
(2500, 6, 13)
Testing Data:
(5000, 11, 13)
(5000, 6, 13)


In [10]:
#print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])


# Build Model

In [11]:
print('Build model...')

############################################
##### Build your own model here ############
model = Sequential()

# "Encode" the input sequence using an RNN, input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))

# the decoder RNN's input
model.add(layers.RepeatVector(DIGITS + 1))

# The decoder RNN  with a single layer.
for _ in range(LAYERS):
    # all the outputs in the form of (num_samples, timesteps, output_dim). 
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))


model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))

############################################
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()


Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 6, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 6, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 6, 13)             1677      
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


# Training

In [12]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # visualizing validation set prediction errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)



--------------------------------------------------
Iteration 0
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [==============================] - 10s 427us/step - loss: 1.9394 - acc: 0.3325 - val_loss: 1.7922 - val_acc: 0.3652
Q        7-59 T 88     ☒ 18    
Q     773+672 T 653    ☒ -46   
Q     6+60303 T 30312  ☒ -446  
Q       1+666 T 667    ☒ 11    
Q       52+09 T 115    ☒ 18    
Q      9-3743 T 3464   ☒ -48   
Q       54+55 T 100    ☒ 15    
Q     46-3916 T 6129   ☒ -466  
Q  31853-8846 T -29325 ☒ -4488 
Q   716-33209 T 89616  ☒ -446  

--------------------------------------------------
Iteration 1
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [==============================] - 9s 400us/step - loss: 1.7092 - acc: 0.3834 - val_loss: 1.6425 - val_acc: 0.3912
Q      4391+3 T 1937   ☒ 180   
Q    064-3097 T 7443   ☒ 1155  
Q      3726-3 T -6270  ☒ -4444 
Q    2762+456 T 3326   ☒ 1005  
Q       21-81 T 6      ☒ -1    
Q     1566+16 T 671

22500/22500 [==============================] - 10s 437us/step - loss: 1.2516 - acc: 0.5345 - val_loss: 1.2476 - val_acc: 0.5315
Q  4892+89355 T 58382  ☒ 52000 
Q  7517+39664 T 53850  ☒ 46600 
Q      33+987 T 822    ☒ 146   
Q       886+9 T 697    ☒ 687   
Q   6911+7454 T 5743   ☒ 5100  
Q    58-16883 T 38776  ☒ 38880 
Q     49215+0 T 51294  ☒ 55500 
Q      8-7418 T 8139   ☒ 8187  
Q        11+0 T 11     ☑ 11    
Q  98846+4615 T 70053  ☒ 10000 

--------------------------------------------------
Iteration 14
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [==============================] - 9s 395us/step - loss: 1.2159 - acc: 0.5477 - val_loss: 1.2233 - val_acc: 0.5447
Q       325+7 T 530    ☒ 522   
Q     1+46403 T 30465  ☒ 30000 
Q       37+04 T 113    ☒ 112   
Q 24874+21267 T 124054 ☒ 13099 
Q       4-819 T 914    ☒ 911   
Q      4524-0 T -4254  ☒ -4244 
Q  80918+0922 T 84198  ☒ 30016 
Q  25206+0681 T 62112  ☒ 33687 
Q    92452-85 T -25371 ☒ -22455
Q        06+4

22500/22500 [==============================] - 12s 535us/step - loss: 1.0013 - acc: 0.6281 - val_loss: 1.0509 - val_acc: 0.6029
Q    459+2621 T 2216   ☒ 2076  
Q     8+10263 T 36209  ☒ 36216 
Q   2576-1665 T -1091  ☒ -11   
Q    04+67933 T 34016  ☒ 33955 
Q      13-439 T 903    ☒ 822   
Q       0-756 T 657    ☒ 659   
Q       18-19 T 10     ☒ -     
Q    9896-764 T -6522  ☒ -6305 
Q  52653+4683 T 39489  ☒ 37777 
Q     18-7115 T 5036   ☒ 5017  

--------------------------------------------------
Iteration 27
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [==============================] - 13s 587us/step - loss: 0.9929 - acc: 0.6321 - val_loss: 1.0432 - val_acc: 0.6064
Q      031+54 T 175    ☒ 158   
Q     6-66707 T 70760  ☒ 70663 
Q    77813+71 T 31894  ☒ 31805 
Q       66-33 T -33    ☒ -43   
Q     2+59014 T 41097  ☒ 40095 
Q 75536-23428 T 18875  ☒ 12229 
Q       77-98 T 12     ☒ 1     
Q      1-5734 T 4374   ☒ 4375  
Q     169+678 T 1837   ☒ 1754  
Q    298+118

22500/22500 [==============================] - 15s 659us/step - loss: 0.8807 - acc: 0.6771 - val_loss: 0.9926 - val_acc: 0.6261
Q      158-25 T -799   ☒ -893  
Q   6833+7053 T 6893   ☒ 7713  
Q     3+64935 T 53949  ☒ 53954 
Q       71+17 T 88     ☒ 96    
Q     46+0868 T 8744   ☒ 8714  
Q    9152+076 T 3189   ☒ 3074  
Q      095-83 T -552   ☒ -534  
Q     07965-0 T -56970 ☒ -56960
Q     05-4036 T 6254   ☒ 6399  
Q      238-88 T -744   ☒ -734  

--------------------------------------------------
Iteration 40
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [==============================] - 16s 725us/step - loss: 0.8712 - acc: 0.6802 - val_loss: 0.9848 - val_acc: 0.6305
Q     888+281 T 1070   ☒ 1087  
Q     6493-48 T -3862  ☒ -3866 
Q     74-9072 T 2662   ☒ 2667  
Q     5628-11 T -8254  ☒ -8256 
Q     05-4036 T 6254   ☒ 6394  
Q     41539-4 T -93510 ☒ -93502
Q  9515+40691 T 24763  ☒ 25737 
Q    35-52196 T 69072  ☒ 69165 
Q   178+44303 T 31215  ☒ 31857 
Q      1461-

22500/22500 [==============================] - 17s 771us/step - loss: 0.7861 - acc: 0.7144 - val_loss: 0.9745 - val_acc: 0.6389
Q    0933+758 T 4247   ☒ 4338  
Q   33934-387 T -43150 ☒ -43566
Q 11587+87889 T 177389 ☒ 107111
Q     8+53879 T 97843  ☒ 97730 
Q 62616+05839 T 155476 ☒ 178544
Q     2726-41 T -6258  ☒ -6267 
Q     05534-2 T -43548 ☒ -43528
Q       782-7 T -280   ☒ -270  
Q   27701-522 T -10547 ☒ -10642
Q    8676-326 T -6145  ☒ -6005 

--------------------------------------------------
Iteration 53
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [==============================] - 16s 731us/step - loss: 0.7779 - acc: 0.7188 - val_loss: 0.9787 - val_acc: 0.6379
Q       4-994 T 495    ☒ 497   
Q     776-297 T 115    ☒ 197   
Q        12+0 T 21     ☒ 233   
Q   1641-9444 T 2988   ☒ 3000  
Q   851-17255 T 55113  ☒ 55131 
Q 99226-84362 T -35951 ☒ -10119
Q     4+89969 T 97002  ☒ 96004 
Q   7951-7319 T 7540   ☒ 8302  
Q    79+73233 T 33334  ☒ 33425 
Q    05914+8

22500/22500 [==============================] - 17s 742us/step - loss: 0.6993 - acc: 0.7508 - val_loss: 0.9853 - val_acc: 0.6464
Q     0-15111 T 11151  ☒ 11150 
Q     5-47463 T 36469  ☒ 36448 
Q     6+60303 T 30312  ☒ 30263 
Q     31-4719 T 9161   ☒ 9169  
Q       3+016 T 613    ☒ 614   
Q     6454-89 T -4448  ☒ -4358 
Q    872-6345 T 5158   ☒ 4262  
Q    3849+747 T 10230  ☒ 10384 
Q     8+85308 T 80366  ☒ 80395 
Q   17388+174 T 88842  ☒ 89270 

--------------------------------------------------
Iteration 66
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [==============================] - 17s 740us/step - loss: 0.6937 - acc: 0.7536 - val_loss: 0.9961 - val_acc: 0.6458
Q  4731+53856 T 67209  ☒ 67001 
Q  8265+48717 T 77412  ☒ 78055 
Q      61+614 T 432    ☒ 438   
Q     03+4751 T 1604   ☒ 1616  
Q    0203-486 T -2336  ☒ -2435 
Q      8773-6 T -3772  ☒ -3762 
Q 22978+51909 T 178837 ☒ 175994
Q       251-4 T -148   ☒ -149  
Q      9-9925 T 5290   ☒ 5298  
Q     345-27

22500/22500 [==============================] - 13s 590us/step - loss: 0.5610 - acc: 0.8067 - val_loss: 1.0884 - val_acc: 0.6477
Q   568-59669 T 95830  ☒ 96093 
Q   6617+2321 T 8398   ☒ 8986  
Q    6493-276 T -3274  ☒ -2759 
Q      91-987 T 770    ☒ 751   
Q    8676-326 T -6145  ☒ -6099 
Q   5005+7112 T 7122   ☒ 6113  
Q     2+98587 T 78591  ☒ 78581 
Q     07204+6 T 40276  ☒ 40282 
Q      2361-4 T -1628  ☒ -1630 
Q     05-2707 T 7022   ☒ 7012  

--------------------------------------------------
Iteration 88
Train on 22500 samples, validate on 2500 samples
Epoch 1/1
22500/22500 [==============================] - 13s 574us/step - loss: 0.5264 - acc: 0.8195 - val_loss: 1.1302 - val_acc: 0.6428
Q       65+41 T 70     ☒ 69    
Q  61582+7834 T 32903  ☒ 39227 
Q       1-253 T 351    ☒ 422   
Q      172+86 T 339    ☒ 350   
Q  80918+0922 T 84198  ☒ 12136 
Q    046+8816 T 6828   ☒ 7316  
Q   6911+7454 T 5743   ☒ 5617  
Q    13+86168 T 86199  ☒ 86226 
Q       7+536 T 642    ☒ 650   
Q     5548-4

# Testing

In [13]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 

preds = model.predict_classes(test_x, verbose=0)

i = 0
count = 0
while i < len(preds) :
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)

    print('Q', q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        count += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
    i += 1
    
testing_acc = (count/len(test_y))*100
print('-'*60)
print('-'*30, colors.ok + str(testing_acc) + "%" + colors.close, '-'*30)
print('-'*60)
#####################################################
    

MSG : Prediction
Q 9+24        T 33     ☑ 33    
Q 567+702     T 1269   ☒ 1287  
Q 631-5       T 626    ☒ 628   
Q 71+552      T 623    ☒ 615   
Q 304+9413    T 9717   ☒ 9894  
Q 18+36542    T 36560  ☒ 365545
Q 22-84       T -62    ☒ -55   
Q 82151+313   T 82464  ☒ 82490 
Q 9-12312     T -12303 ☒ -12339
Q 93-9834     T -9741  ☒ -9767 
Q 59538-409   T 59129  ☒ 59336 
Q 6340-823    T 5517   ☒ 5511  
Q 949+2       T 951    ☒ 950   
Q 7+423       T 430    ☒ 431   
Q 91-688      T -597   ☒ -502  
Q 564-27      T 537    ☒ 535   
Q 772-8       T 764    ☒ 765   
Q 71+7703     T 7774   ☒ 7794  
Q 19-9        T 10     ☒ 12    
Q 140+49196   T 49336  ☒ 49220 
Q 44595-98038 T -53443 ☒ -33333
Q 394+94      T 488    ☒ 482   
Q 557+458     T 1015   ☒ 1082  
Q 25228-5     T 25223  ☒ 25232 
Q 766+1764    T 2530   ☒ 2595  
Q 13-67       T -54    ☒ -57   
Q 451+96471   T 96922  ☒ 95188 
Q 82290-1633  T 80657  ☒ 79777 
Q 622-44      T 578    ☒ 575   
Q 289+8700    T 8989   ☒ 8882  
Q 9423+52     T 9475   

Q 17+972      T 989    ☑ 989   
Q 885+70246   T 71131  ☒ 71646 
Q 2-648       T -646   ☑ -646  
Q 254-75      T 179    ☒ 161   
Q 52+407      T 459    ☒ 465   
Q 4588-7160   T -2572  ☒ -2639 
Q 264-183     T 81     ☒ 439   
Q 2970+8862   T 11832  ☒ 11714 
Q 57817-43115 T 14702  ☒ 19136 
Q 2-960       T -958   ☒ -959  
Q 14285+0     T 14285  ☒ 14490 
Q 4272-69     T 4203   ☒ 4217  
Q 0-7651      T -7651  ☒ -76501
Q 15354-4     T 15350  ☒ 15540 
Q 90+5757     T 5847   ☒ 5849  
Q 6396+6890   T 13286  ☒ 13705 
Q 0+413       T 413    ☒ 414   
Q 39935-88    T 39847  ☒ 39398 
Q 82-312      T -230   ☒ -233  
Q 1170-3      T 1167   ☒ 1164  
Q 4522-75884  T -71362 ☒ -71655
Q 7369+1      T 7370   ☒ 7369  
Q 63-37612    T -37549 ☒ -37654
Q 729+50591   T 51320  ☒ 55317 
Q 9294-35391  T -26097 ☒ -26827
Q 148-70      T 78     ☒ 836   
Q 28-86       T -58    ☑ -58   
Q 895-3365    T -2470  ☒ -2500 
Q 67-2015     T -1948  ☒ -1939 
Q 833+92      T 925    ☒ 922   
Q 25872+6088  T 31960  ☒ 32827 
Q 370-37

Q 45-78       T -33    ☒ -39   
Q 745+3       T 748    ☒ 747   
Q 2911-455    T 2456   ☒ 2454  
Q 67-25803    T -25736 ☒ -25895
Q 31-53       T -22    ☒ -17   
Q 1046-4      T 1042   ☒ 1041  
Q 13614-4386  T 9228   ☒ 11511 
Q 74507+7383  T 81890  ☒ 83205 
Q 41+5202     T 5243   ☒ 5254  
Q 0-7730      T -7730  ☒ -7713 
Q 1793-8      T 1785   ☒ 1790  
Q 31636+85687 T 117323 ☒ 10072 
Q 2-9962      T -9960  ☒ -9966 
Q 68126+1     T 68127  ☒ 68116 
Q 0+7482      T 7482   ☑ 7482  
Q 329-9767    T -9438  ☒ -9537 
Q 68+5474     T 5542   ☒ 5733  
Q 61722+9674  T 71396  ☒ 70680 
Q 33+29       T 62     ☑ 62    
Q 4198+988    T 5186   ☒ 5009  
Q 7-84932     T -84925 ☒ -84920
Q 53906-1     T 53905  ☒ 53901 
Q 9+2177      T 2186   ☒ 2185  
Q 30935-4700  T 26235  ☒ 27769 
Q 142-3111    T -2969  ☒ -2992 
Q 4256-99940  T -95684 ☒ -93533
Q 1+12        T 13     ☑ 13    
Q 810+459     T 1269   ☒ 1281  
Q 5-94483     T -94478 ☒ -94531
Q 138+342     T 480    ☒ 562   
Q 971-5       T 966    ☒ 975   
Q 7366-2

Q 67039-9     T 67030  ☒ 67084 
Q 8415-2      T 8413   ☒ 8419  
Q 0-77712     T -77712 ☒ -77710
Q 7809-8643   T -834   ☒ -111  
Q 98230+64    T 98294  ☒ 98286 
Q 18-22709    T -22691 ☒ -22798
Q 59-1        T 58     ☑ 58    
Q 60126+3     T 60129  ☒ 60021 
Q 236-37      T 199    ☒ 212   
Q 2270+43     T 2313   ☒ 2390  
Q 8126+54     T 8180   ☒ 8167  
Q 1+763       T 764    ☑ 764   
Q 238-41725   T -41487 ☒ -40459
Q 6407+20     T 6427   ☒ 6413  
Q 3+38928     T 38931  ☒ 38961 
Q 0-554       T -554   ☒ -553  
Q 788-6877    T -6089  ☒ -60095
Q 6236-0      T 6236   ☑ 6236  
Q 4+526       T 530    ☑ 530   
Q 56896-32829 T 24067  ☒ 21000 
Q 54421-16022 T 38399  ☒ 42220 
Q 63-582      T -519   ☒ -503  
Q 311+99399   T 99710  ☒ 90322 
Q 362-6519    T -6157  ☒ -6045 
Q 7249+191    T 7440   ☒ 7528  
Q 12-90644    T -90632 ☒ -90639
Q 30210-99    T 30111  ☒ 30248 
Q 23+4        T 27     ☑ 27    
Q 827+594     T 1421   ☒ 1380  
Q 623-8       T 615    ☒ 616   
Q 34-24       T 10     ☒ 1     
Q 77894-

Q 60-88610    T -88550 ☒ -88684
Q 1145+95329  T 96474  ☒ 96887 
Q 620+2       T 622    ☒ 623   
Q 59-5047     T -4988  ☒ -4059 
Q 40366-91745 T -51379 ☒ -46511
Q 65450+1491  T 66941  ☒ 66644 
Q 80-45       T 35     ☒ 36    
Q 28621-11129 T 17492  ☒ 27787 
Q 600+2358    T 2958   ☒ 3976  
Q 41-9897     T -9856  ☒ -9944 
Q 9342-1875   T 7467   ☒ 8294  
Q 3-637       T -634   ☒ -632  
Q 41582+33321 T 74903  ☒ 56842 
Q 50325+898   T 51223  ☒ 51133 
Q 66-3408     T -3342  ☒ -3338 
Q 9176+73147  T 82323  ☒ 80658 
Q 68150+2     T 68152  ☒ 68164 
Q 51963+81    T 52044  ☒ 52985 
Q 4-607       T -603   ☒ -602  
Q 42-40       T 2      ☒ 1     
Q 9412+5      T 9417   ☒ 9414  
Q 72325-2     T 72323  ☒ 72327 
Q 7-2661      T -2654  ☒ -2610 
Q 99+5188     T 5287   ☒ 5279  
Q 25682-11077 T 14605  ☒ 15464 
Q 915+3788    T 4703   ☒ 4775  
Q 2043+51149  T 53192  ☒ 66177 
Q 78+44758    T 44836  ☒ 44821 
Q 7902-7705   T 197    ☒ 136   
Q 6167-778    T 5389   ☒ 5101  
Q 4551-99     T 4452   ☒ 4455  
Q 76628-

...

Q 77-92738    T -92661 ☒ -92775
Q 446+57      T 503    ☒ 505   
Q 925+850     T 1775   ☒ 1879  
Q 6+7475      T 7481   ☒ 7452  
Q 1313-95290  T -93977 ☒ -95005
Q 753-60761   T -60008 ☒ -65988
Q 649+1858    T 2507   ☒ 2355  
Q 87+2423     T 2510   ☒ 2412  
Q 1502-617    T 885    ☒ 3117  
Q 96+3        T 99     ☒ 90    
Q 798+8283    T 9081   ☒ 8102  
Q 568+103     T 671    ☒ 610   
Q 5550+61452  T 67002  ☒ 60377 
Q 986+178     T 1164   ☒ 1050  
Q 5-58961     T -58956 ☒ -59953
Q 14731+4231  T 18962  ☒ 28186 
Q 71-754      T -683   ☒ -672  
Q 21-59       T -38    ☒ -34   
Q 722-183     T 539    ☒ 549   
Q 32608+43428 T 76036  ☒ 57650 
Q 735-99311   T -98576 ☒ -98645
Q 7543+23     T 7566   ☒ 7576  
Q 349-16855   T -16506 ☒ -16511
Q 798-576     T 222    ☒ 202   
Q 91572+70863 T 162435 ☒ 159555
Q 86228-47001 T 39227  ☒ 38128 
Q 311-3       T 308    ☒ 317   
Q 86068-3     T 86065  ☒ 86076 
Q 86-17       T 69     ☒ 70    
Q 3-721       T -718   ☒ -715  
Q 0-8368      T -8368  ☒ -8367 
Q 858+68

Q 46258+33    T 46291  ☒ 46231 
Q 0+792       T 792    ☒ 793   
Q 7444+333    T 7777   ☒ 7876  
Q 17-26       T -9     ☒ -6    
Q 0+68084     T 68084  ☒ 60081 
Q 1-43        T -42    ☒ -41   
Q 54+5        T 59     ☒ 58    
Q 82+70574    T 70656  ☒ 70662 
Q 230+47974   T 48204  ☒ 47068 
Q 9-90        T -81    ☒ -84   
Q 47+58656    T 58703  ☒ 58707 
Q 6-7186      T -7180  ☒ -7177 
Q 42806-6147  T 36659  ☒ 34257 
Q 86+91232    T 91318  ☒ 91286 
Q 7267-657    T 6610   ☒ 6502  
Q 9629-7731   T 1898   ☒ 1048  
Q 3+7667      T 7670   ☒ 7775  
Q 389-2       T 387    ☑ 387   
Q 7-750       T -743   ☒ -744  
Q 48+37       T 85     ☑ 85    
Q 36648-903   T 35745  ☒ 35765 
Q 26231+6184  T 32415  ☒ 22096 
Q 35062-90    T 34972  ☒ 35018 
Q 38+66089    T 66127  ☒ 66197 
Q 3-94363     T -94360 ☒ -94334
Q 37489+991   T 38480  ☒ 48352 
Q 78-67       T 11     ☒ 10    
Q 561-3       T 558    ☒ 559   
Q 34-28       T 6      ☒ 5     
Q 27022+460   T 27482  ☒ 27362 
Q 86+8826     T 8912   ☒ 8954  
Q 44827+

Q 2122+0      T 2122   ☒ 2113  
Q 93+997      T 1090   ☒ 1088  
Q 3809-5975   T -2166  ☒ -3631 
Q 308+15561   T 15869  ☒ 15687 
Q 24571-923   T 23648  ☒ 23782 
Q 24191-7666  T 16525  ☒ 15522 
Q 19359-1281  T 18078  ☒ 27594 
Q 7750+684    T 8434   ☒ 8315  
Q 8+440       T 448    ☒ 447   
Q 205+512     T 717    ☒ 736   
Q 70713-27    T 70686  ☒ 70664 
Q 76950-0     T 76950  ☒ 76954 
Q 22998+525   T 23523  ☒ 23564 
Q 61-2507     T -2446  ☒ -2448 
Q 303+89      T 392    ☒ 397   
Q 4+5039      T 5043   ☒ 5044  
Q 4429+93     T 4522   ☒ 4428  
Q 4823-47575  T -42752 ☒ -43919
Q 39465+9     T 39474  ☒ 39456 
Q 60+6        T 66     ☑ 66    
Q 91942+575   T 92517  ☒ 92550 
Q 9848+59284  T 69132  ☒ 17823 
Q 4342+28858  T 33200  ☒ 33221 
Q 181+4685    T 4866   ☒ 5016  
Q 26+13126    T 13152  ☒ 13295 
Q 87768-57088 T 30680  ☒ 13588 
Q 653+64      T 717    ☒ 714   
Q 84930-531   T 84399  ☒ 84979 
Q 2734-561    T 2173   ☒ 2657  
Q 96+862      T 958    ☒ 984   
Q 7441+2      T 7443   ☒ 7442  
Q 7+1223

Q 5097+313    T 5410   ☒ 5448  
Q 77-72       T 5      ☒ 6     
Q 996+8719    T 9715   ☒ 8887  
Q 0-5735      T -5735  ☑ -5735 
Q 633+5       T 638    ☒ 647   
Q 66-23554    T -23488 ☒ -23214
Q 72980+9     T 72989  ☒ 72900 
Q 3618-41503  T -37885 ☒ -39347
Q 74-24698    T -24624 ☒ -24441
Q 5974+8321   T 14295  ☒ 13721 
Q 56217-9530  T 46687  ☒ 44757 
Q 1-9045      T -9044  ☒ -90401
Q 7801+356    T 8157   ☒ 7851  
Q 29+27786    T 27815  ☒ 27413 
Q 4709+3      T 4712   ☒ 4719  
Q 58+51273    T 51331  ☒ 51482 
Q 481+6187    T 6668   ☒ 6677  
Q 496-41      T 455    ☒ 443   
Q 56800+39089 T 95889  ☒ 10014 
Q 23214+8984  T 32198  ☒ 29256 
Q 4916+6      T 4922   ☒ 4913  
Q 84835+3697  T 88532  ☒ 97017 
Q 6227+7353   T 13580  ☒ 13885 
Q 41+21815    T 21856  ☒ 21911 
Q 3976+6156   T 10132  ☒ 1133  
Q 35+74       T 109    ☒ 91    
Q 7352+3208   T 10560  ☒ 80283 
Q 14-99587    T -99573 ☒ -99599
Q 9297-7992   T 1305   ☒ 3249  
Q 34+2768     T 2802   ☒ 2812  
Q 64301-5     T 64296  ☒ 64305 
Q 29210+

Q 5-59        T -54    ☑ -54   
Q 58+4605     T 4663   ☑ 4663  
------------------------------------------------------------
------------------------------ 5.4399999999999995% ------------------------------
------------------------------------------------------------
